In [8]:
#General Purpose
import pandas as pd
import numpy as np
import re
import string
import spacy
import random
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


#Classification and Metrics
from sklearn.model_selection import GridSearchCV, train_test_split, PredefinedSplit
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.base import BaseEstimator, TransformerMixin
from gensim.models.doc2vec import TaggedDocument


from sklearn import metrics

%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
clean_reviews = pd.read_csv('/content/drive/MyDrive/modelling/clean_reviews_dataset.csv')

In [10]:
clean_reviews = clean_reviews.dropna(subset=['cleaned_reviews_reduced_words'])
clean_reviews = clean_reviews[clean_reviews['cleaned_reviews_reduced_words'].apply(lambda x: len(str(x)) > 0)]

In [11]:
mlb = MultiLabelBinarizer()
binarised_labels = mlb.fit_transform(clean_reviews['Cuisines'])
clean_reviews['labels'] = binarised_labels.tolist()

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    clean_reviews['cleaned_reviews_reduced_words'], binarised_labels, test_size=0.2, random_state=42, stratify=binarised_labels)

X_train = np.array([str(x) for x in X_train])

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [14]:
X_train_full = np.concatenate((X_train, X_val), axis=0)
y_train_full = np.concatenate((y_train, y_val), axis=0)

split_index = [-1] * len(X_train) + [0] * len(X_val)
predefined_split = PredefinedSplit(test_fold=split_index)

In [15]:
bigram_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1), token_pattern=r"(?u)\b\w+\b")
bigram_bow_vectorizer = CountVectorizer(ngram_range=(1,1), token_pattern=r"(?u)\b\w+\b")

In [16]:
scores = ["accuracy", "precision_weighted", "recall_weighted", "f1_weighted"]

#### One Vs Rest

In [25]:
pipeline_chain_lr = Pipeline([
    ('vectorizer', bigram_tfidf_vectorizer),
    ('classifier', OneVsRestClassifier(LogisticRegression(random_state=42)))])

param_grid_ovr_lr = {
    'vectorizer': [bigram_tfidf_vectorizer, bigram_bow_vectorizer],
    'classifier__estimator__solver': ['saga', 'lbfgs'],
    'classifier__estimator__C': [0.1, 1, 10],
    'classifier__estimator__penalty': [None, 'l2'],
    'classifier__estimator__class_weight': [None, 'balanced'],
    'classifier__estimator__max_iter': [100, 200],
    'classifier__estimator__fit_intercept': [True, False],
    'classifier__estimator__tol': [1e-4]}

grid_search = GridSearchCV(
    estimator=pipeline_chain_lr,
    param_grid=param_grid_ovr_lr,
    cv = predefined_split,
    scoring=scores,
    refit="f1_weighted",
    verbose=4)

grid_result = grid_search.fit(X_train_full, y_train_full)

Fitting 1 folds for each of 192 candidates, totalling 192 fits
[CV 1/1] END classifier__estimator__C=0.1, classifier__estimator__class_weight=None, classifier__estimator__fit_intercept=True, classifier__estimator__max_iter=100, classifier__estimator__penalty=None, classifier__estimator__solver=saga, classifier__estimator__tol=0.0001, vectorizer=TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b'); accuracy: (test=0.136) f1_weighted: (test=0.855) precision_weighted: (test=0.859) recall_weighted: (test=0.855) total time=  20.6s
[CV 1/1] END classifier__estimator__C=0.1, classifier__estimator__class_weight=None, classifier__estimator__fit_intercept=True, classifier__estimator__max_iter=100, classifier__estimator__penalty=None, classifier__estimator__solver=saga, classifier__estimator__tol=0.0001, vectorizer=CountVectorizer(token_pattern='(?u)\\b\\w+\\b'); accuracy: (test=0.156) f1_weighted: (test=0.858) precision_weighted: (test=0.870) recall_weighted: (test=0.867) total time=  21.3s
[CV 1/1] 

In [28]:
print("Best Parameters (Logistic Regression One Vs Rest): ", grid_result.best_params_)

Best Parameters (Logistic Regression One Vs Rest):  {'classifier__estimator__C': 10, 'classifier__estimator__class_weight': None, 'classifier__estimator__fit_intercept': True, 'classifier__estimator__max_iter': 100, 'classifier__estimator__penalty': 'l2', 'classifier__estimator__solver': 'saga', 'classifier__estimator__tol': 0.0001, 'vectorizer': TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b')}


In [29]:
for score in scores:
    mean_score = grid_result.cv_results_[f'mean_test_{score}'][grid_result.best_index_]
    print(f"Logistic Regression One Vs Rest: \n{score} = {round(mean_score, 3)}")

Logistic Regression One Vs Rest: 
accuracy = 0.17
Logistic Regression One Vs Rest: 
precision_weighted = 0.879
Logistic Regression One Vs Rest: 
recall_weighted = 0.868
Logistic Regression One Vs Rest: 
f1_weighted = 0.863


#### Classifier Chain

In [ ]:
pipeline_chain_lr = Pipeline([
    ('vectorizer', bigram_tfidf_vectorizer),
    ('classifier', ClassifierChain(LogisticRegression(random_state=42)))])

param_grid_chain_lr = {
    'vectorizer': [bigram_tfidf_vectorizer, bigram_bow_vectorizer],
    'classifier__order': [None, 'random'],

    'classifier__base_estimator__solver': ['saga', 'lbfgs'],
    'classifier__base_estimator__C': [1, 10],
    'classifier__base_estimator__penalty': ['l2'],
    'classifier__base_estimator__class_weight': [None, 'balanced'],
    'classifier__base_estimator__max_iter': [100, 200],
    'classifier__base_estimator__fit_intercept': [True],
    'classifier__base_estimator__tol': [1e-4]}

grid_search_chain = GridSearchCV(
    estimator=pipeline_chain_lr,
    param_grid=param_grid_chain_lr,
    scoring=scores,
    cv = predefined_split,
    refit="f1_weighted",
    error_score='raise',
    verbose=4)

grid_result_chain = grid_search_chain.fit(X_train_full, y_train_full)

In [ ]:
print("Best Parameters (Logistic Regression Classifier Chain): ", grid_result_chain.best_params_)

In [ ]:
for score in scores:
    mean_score = grid_result_chain.cv_results_[f'mean_test_{score}'][grid_result_chain.best_index_]
    print(f"Logistic Regression One Vs Rest: \n{score} = {round(mean_score, 3)}")